In [1]:
from qcutils.credential import load_provider
from qcutils.benchmarks import QASMBenchmark
from qcutils.pulse import CircAmpTimeSeries, BasisPulse
from qcutils.pulse import CircAmpTimeSeries
from qcutils.pulse.amptimeseries.metric import dist, min_circ_dist_in_list
from qcutils.circuit.library import deutsch_jozsa, bernstein_vazirani, grover

from qiskit import transpile


provider = load_provider()
backend = provider.get_backend("ibm_lagos")

In [6]:
# save the current basis pulse library

bp = BasisPulse(backend)
bp.save_basis_amp_time_series_list(f"./data/basis_pulse_{backend.configuration().backend_name}.pickle")

In [2]:
def generate_binary_numbers(n):
    result = []
    for i in range(2**n):
        binary_str = bin(i)[2:]
        binary_str = "0"*(n-len(binary_str)) + binary_str
        result.append(binary_str)
    return result

def circ_list(algo, num_qubits, backend, **transpile_args):
    if algo == "dj":
        b_str_list = generate_binary_numbers(num_qubits - 1)
        # function_type_list = ["constant", "balanced"]
        # circ_list = [deutsch_jozsa(backend, num_qubits, qubits_list=range(num_qubits), function_type=function_type, b_str=b_str, measure=False) for function_type in function_type_list for b_str in b_str_list]
        circ_list = [deutsch_jozsa(num_qubits, qubits_list=range(num_qubits), function_type="balanced", b_str=b_str, measure=False) for b_str in b_str_list]
    elif algo == "bv":
        hidden_str_list = generate_binary_numbers(num_qubits - 1)
        circ_list = [bernstein_vazirani(num_qubits, qubits_list=range(num_qubits), hidden_str=hidden_str, measure=False) for hidden_str in hidden_str_list]
    elif algo == "gs":
        search_state_list = generate_binary_numbers(num_qubits)
        circ_list = [grover(num_qubits, qubits_list=range(num_qubits), search_state=search_state, measure=False) for search_state in search_state_list]
    
    transpiled_circ_list = transpile(circ_list, backend, initial_layout=range(num_qubits), **transpile_args)
    return transpiled_circ_list

def result(num_qubits_list, backend, algo_list = ["bv", "dj", "gs"], **transpile_args):
    print("Algorithm" + "\t" * int(len(num_qubits_list)/2) + "Number of Qubits")
    print("\t\t" + "\t".join([str(num_qubits) for num_qubits in num_qubits_list]))

    for algo in algo_list:
        min_dist_list = [min_circ_dist_in_list(circ_list(algo, num_qubits, backend, **transpile_args), backend, normalize=True)[0] for num_qubits in num_qubits_list]
        print(f"{algo}\t\t" + "\t".join(["{:.3f}".format(min_dist) for min_dist in min_dist_list]))

In [4]:
result(range(2, 8), backend,  algo_list =["bv", "dj"], seed_transpiler=0)

Algorithm			Number of Qubits
		2	3	4	5	6	7
bv		0.999	0.298	0.074	0.062	0.068	0.064


In [5]:
result(range(2, 8), backend,  algo_list = ["dj"], seed_transpiler=0)

Algorithm			Number of Qubits
		2	3	4	5	6	7
dj		0.000	0.000	0.000	0.000	0.000	0.000


In [3]:
result(range(1, 7), backend, algo_list=["gs"], seed_transpiler=0)

Algorithm			Number of Qubits
		1	2	3	4	5	6
gs		0.000	0.000	0.000	0.000	0.000	0.000
